In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [6]:
all_set = pd.read_csv('./data/[회귀]all_2차_전처리_완료.csv')

In [7]:
df17 = all_set[all_set['연도'] == 2017]
df18 = all_set[all_set['연도'] == 2018]
df19 = all_set[all_set['연도'] == 2019]

In [8]:
all_set['입출구명'].unique()

array(['구의문', '후문', '정문', '사전무인', '서버실'], dtype=object)

# 2017년 입장문별 데이터 분리

In [9]:
df19_guiient = df19[df19['입출구명'] == '구의문']
df19_backent = df19[df19['입출구명'] == '후문']
df19_frontent = df19[df19['입출구명'] == '정문']

In [10]:
df19_guiient

,진입시각,진출시각,입출구명,고객구분,차종,주차시간(분),연도,in_year,in_month,in_date,in_hour,in_minute,in_second,out_year,out_month,out_date,out_hour,out_minute,out_second
516053,2019-01-10 13:06:23,2019-01-10 17:49:44,구의문,일반,소형,284,2019,2019,1,10,13,6,23,2019,1,10,17,49,44
516125,2019-01-10 15:52:33,2019-01-10 16:51:59,구의문,일반,소형,60,2019,2019,1,10,15,52,33,2019,1,10,16,51,59
516129,2019-01-10 15:58:41,2019-01-10 16:51:14,구의문,일반,소형,53,2019,2019,1,10,15,58,41,2019,1,10,16,51,14
516134,2019-01-10 16:09:02,2019-01-10 18:44:38,구의문,일반,소형,156,2019,2019,1,10,16,9,2,2019,1,10,18,44,38
516137,2019-01-10 16:10:00,2019-01-10 18:04:00,구의문,일반,소형,114,2019,2019,1,10,16,10,0,2019,1,10,18,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897831,2019-10-24 09:10:05,2019-10-24 10:26:58,구의문,일반,소형,77,2019,2019,10,24,9,10,5,2019,10,24,10,26,58
897833,2019-10-24 09:14:31,2019-10-24 09:58:05,구의문,일반,소형,44,2019,2019,10,24,9,14,31,2019,10,24,9,58,5
897837,2019-10-24 09:36:04,2019-10-24 10:25:32,구의문,일반,소형,50,2019,2019,10,24,9,36,4,2019,10,24,10,25,32
897839,2019-10-24 10:00:30,2019-10-24 10:16:41,구의문,일반,소형,16,2019,2019,10,24,10,0,30,2019,10,24,10,16,41


# 카운팅 데이터프레임 만들기

In [11]:
counting_df_2019 = pd.DataFrame()
counting_df_2019['datetime'] = pd.date_range("2019-01-01", "2019-11-13", freq='H')
counting_df_2019.dtypes

datetime    datetime64[ns]
dtype: object

# [입장문별] 시각 기준 카운팅

In [14]:
def counting_hourly(counting_df, row, counter):
    compare_month = counting_df['datetime'].dt.month == row['in_month']
    compare_date = counting_df['datetime'].dt.day == row['in_date']
    compare_in_hour = counting_df['datetime'].dt.hour >= row['in_hour']
    compare_out_hour = counting_df['datetime'].dt.hour <= row['out_hour']
    
    counter.append(list(counting_df.loc[compare_month & compare_date & compare_in_hour & compare_out_hour]['datetime']))
    pass

In [15]:
def iter_counter(counting_df, df_ent):
    counter = []
    df_ent.apply(lambda row: counting_hourly(counting_df, row, counter), axis=1)
    counter_save = counter.copy()
    counter = [item for item_list in counter for item in item_list]
    counter = pd.DataFrame(counter).rename(columns={0 : 'datetime'})
    counter['count'] = 1
    counter = counter.groupby('datetime').count().reset_index()
    counter_2018_save = pd.merge(counting_df, counter, how='outer', on='datetime').fillna(0)
    return counter_2018_save

In [16]:
# 정문
iter_counter(counting_df_2019, df19_frontent).to_csv('./data/[회귀]실이용객_시각별_차량_카운트_정문_2019.csv', index=False)

In [17]:
# 후문
iter_counter(counting_df_2019, df19_backent).to_csv('./data/[회귀]실이용객_시각별_차량_카운트_후문_2019.csv', index=False)

In [18]:
# 구의문
iter_counter(counting_df_2019, df19_guiient).to_csv('./data/[회귀]실이용객_시각별_차량_카운트_구의문_2019.csv', index=False)